In [ ]:
import pandas as pd  # Importa la libreria pandas per la manipolazione dei dati
import sqlite3       # Importa la libreria sqlite3 per lavorare con database SQLite

conn = sqlite3.connect("azienda_dipendenti_aggiornato.db")  # Crea una connessione al database SQLite

dipendenti_df = pd.DataFrame({  # Crea un DataFrame pandas con i dati dei dipendenti
    'id': range(1, 9),
    'nome': ['Marco', 'Anna', 'Luca', 'Sara', 'Giulia', 'Paolo', 'Elena', 'Franco'],
    'dipartimento': ['IT', 'HR', 'IT', 'Marketing', 'HR', 'IT', 'Marketing', 'Contabilità'],
    'stipendio': [2000, 1800, 2200, 1900, 1750, 2100, 1950, 1850],
    'data_assunzione': [
        '2018-05-10', '2021-06-15', '2019-07-01', '2022-01-10',
        '2017-03-12', '2015-11-30', '2023-04-25', '2016-09-20'
    ]
})

dipendenti_df.to_sql('dipendenti', conn, if_exists='replace', index=False)  # Salva il DataFrame come tabella 'dipendenti' nel database

media_dipartimenti = pd.read_sql_query("""  # Esegue una query SQL per calcolare la media degli stipendi per dipartimento
SELECT dipartimento, AVG(stipendio) AS media_stipendio
FROM dipendenti
GROUP BY dipartimento
""", conn)
print("Media stipendio per dipartimento:")  # Stampa una descrizione
print(media_dipartimenti)                   # Stampa il risultato della query

media_dipartimenti = pd.read_sql_query("""  # Ripete la stessa query di prima (ridondante)
SELECT dipartimento, AVG(stipendio) AS media_stipendio
FROM dipendenti
GROUP BY dipartimento
""", conn)
print("Media stipendio per dipartimento:")  # Stampa una descrizione
print(media_dipartimenti)                   # Stampa il risultato della query

conn.execute("""  # Aggiorna lo stipendio a 2000 per i dipendenti assunti prima del 2020
UPDATE dipendenti
SET stipendio = 2000
WHERE data_assunzione < '2020-01-01'
""")
conn.commit()  # Salva le modifiche nel database

sopra_media_df = pd.read_sql_query("""  # Seleziona i dipendenti con stipendio sopra la media generale (prima versione)
SELECT *
FROM dipendenti
WHERE stipendio > (
    SELECT AVG(stipendio)
    FROM dipendenti
)
""", conn)

media_generale = pd.read_sql_query("""  # Calcola la media generale degli stipendi
SELECT AVG(stipendio) AS media_generale FROM dipendenti
""", conn).iloc[0, 0]

sopra_media_df = pd.read_sql_query(f"""  # Seleziona i dipendenti con stipendio sopra la media generale (usando la variabile)
SELECT * FROM dipendenti
WHERE stipendio > {media_generale}
""", conn)

print("\nDipendenti con stipendio sopra la media generale:")  # Stampa una descrizione
print(sopra_media_df)                                         # Stampa il risultato della query

conn.close()  # Chiude la connessione al database

Media stipendio per dipartimento:
  dipartimento  media_stipendio
0  Contabilità           1850.0
1           HR           1775.0
2           IT           2100.0
3    Marketing           1925.0
Media stipendio per dipartimento:
  dipartimento  media_stipendio
0  Contabilità           1850.0
1           HR           1775.0
2           IT           2100.0
3    Marketing           1925.0

Dipendenti con stipendio sopra la media generale:
   id    nome dipartimento  stipendio data_assunzione
0   1   Marco           IT       2000      2018-05-10
1   3    Luca           IT       2000      2019-07-01
2   5  Giulia           HR       2000      2017-03-12
3   6   Paolo           IT       2000      2015-11-30
4   8  Franco  Contabilità       2000      2016-09-20


In [1]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

conn = sqlite3.connect("ordini_marketplace.db")

ordini_df = pd.DataFrame({
    'id': range(1, 16),
    'cliente': ['Anna', 'Marco', 'Luca', 'Giulia', 'Elena', 'Franco', 'Sara', 'Giorgio', 'Linda', 'Roberto',
                'Anna', 'Marco', 'Giorgio', 'Elena', 'Sara'],
    'prodotto': ['Penna', 'Libro', 'Notebook', 'Zaino', 'Agenda', 'Quaderno', 'Matita', 'Cartella', 'Penna', 'Matita',
                 'Notebook', 'Zaino', 'Agenda', 'Quaderno', 'Libro'],
    'quantità': [1, 2, 1, 3, 2, 5, 2, 1, 4, 1, 1, 2, 1, 3, 1],
    'prezzo_unitario': [1.0, 15.0, 12.0, 20.0, 10.0, 1.5, 0.8, 30.0, 1.0, 0.8, 12.0, 20.0, 10.0, 1.5, 15.0],
    'data_ordine': [(datetime.today() - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(15)]
})

ordini_df.to_sql('ordini', conn, if_exists='replace', index=False)

spesa_clienti = pd.read_sql_query("""
SELECT cliente, SUM(quantità * prezzo_unitario) AS totale_spesa
FROM ordini
GROUP BY cliente
""", conn)

media_generale = spesa_clienti['totale_spesa'].mean()

clienti_sopra_media = spesa_clienti[spesa_clienti['totale_spesa'] > media_generale]
print("\nClienti con spesa superiore alla media:")
print(clienti_sopra_media)

conn.execute("""
DELETE FROM ordini
WHERE quantità * prezzo_unitario < 3
""")
conn.commit()
conn.close()




Clienti con spesa superiore alla media:
   cliente  totale_spesa
3  Giorgio          40.0
4   Giulia          60.0
7    Marco          70.0


In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("prenotazioni_ristorante.db")

prenotazioni_df = pd.DataFrame({
    'id': range(1, 11),
    'cliente': ['Anna', 'Marco', 'Luca', 'Giulia', 'Elena', 'Paolo', 'Sara', 'Franco', 'Linda', 'Giorgio'],
    'data_prenotazione': [
        '2024-07-01', '2024-07-02', '2024-07-02', '2024-07-03', '2024-07-03',
        '2024-07-04', '2024-07-04', '2024-07-05', '2024-07-05', '2024-07-06'
    ],
    'numero_persone': [2, 4, 1, 3, 5, 6, 2, 1, 4, 3],
    'fascia_oraria': ['pranzo', 'cena', 'pranzo', 'cena', 'cena', 'pranzo', 'cena', 'pranzo', 'cena', 'pranzo']
})

prenotazioni_df.to_sql('prenotazioni', conn, if_exists='replace', index=False)

prenotazioni_gruppi = pd.read_sql_query("""
SELECT * FROM prenotazioni
WHERE numero_persone >= 4
""", conn)
print("Prenotazioni con 4 o più persone:")
print(prenotazioni_gruppi)

conn.execute("""
DELETE FROM prenotazioni
WHERE numero_persone < 2
""")
conn.commit()

media_per_fascia = pd.read_sql_query("""
SELECT fascia_oraria, AVG(numero_persone) AS media_persone
FROM prenotazioni
GROUP BY fascia_oraria
""", conn)
print("\nMedia persone per fascia oraria:")
print(media_per_fascia)
conn.close()

